In [1]:
from kiteconnect import KiteConnect, KiteTicker
import logging

In [ ]:
# Configure logger
logging.basicConfig(level=logging.DEBUG)

In [ ]:
# KiteConnect credentials
api_key = "YOUR_API_KEY"
access_token = "YOUR_ACCESS_TOKEN"

# Initialize KiteConnect
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token)

# Initialize KiteTicker
kite_ticker = KiteTicker(api_key, kite.ltp)

In [ ]:
# Trailing stop-loss parameters
trailing_stop_loss_percent = 0.5  # trailing stop-loss percentage
stop_loss_triggered = False

# Function to calculate trailing stop-loss
def calculate_trailing_stop_loss(trigger_price, current_price):
    return trigger_price * (1 - trailing_stop_loss_percent / 100)

In [ ]:
# Function to handle KiteTicker on_ticks event
def on_ticks(ticks):
    global stop_loss_triggered

    # Get the current instrument price
    current_price = ticks[0]["last_price"]

    # Check if stop loss is triggered
    if stop_loss_triggered:
        # Close the trade
        print("Stop loss triggered. Closing the trade...")
        # Place your code to close the trade using KiteConnect's API

        # Unsubscribe from ticker updates
        kite_ticker.unsubscribe([instrument_token])
        return

    # Check if the current price is below the trailing stop-loss trigger price
    if current_price <= trailing_stop_loss_trigger:
        stop_loss_triggered = True
        print("Trailing stop-loss triggered at price:", current_price)

In [ ]:
# Function to handle KiteTicker on_connect event
def on_connect(response):
    # Subscribe to instrument tickers
    global instrument_token, trailing_stop_loss_trigger

    # Example option instrument token
    instrument_token = 12345678

    # Get the instrument details
    instrument_info = kite.ltp(["NSE:" + str(instrument_token)])
    instrument_price = instrument_info["NSE:" + str(instrument_token)]["last_price"]

    # Calculate trailing stop-loss trigger price
    trailing_stop_loss_trigger = calculate_trailing_stop_loss(instrument_price, instrument_price)
    print("Trailing stop-loss trigger price:", trailing_stop_loss_trigger)

    # Subscribe to the instrument ticker
    kite_ticker.subscribe([instrument_token])

In [ ]:
# Function to handle KiteTicker on_close event
def on_close(code, reason):
    print("KiteTicker connection closed:", code, reason)

In [ ]:
# Function to handle KiteTicker on_error event
def on_error(code, reason):
    print("KiteTicker connection error:", code, reason)

In [ ]:
# Configure KiteTicker event handlers
kite_ticker.on_ticks = on_ticks
kite_ticker.on_connect = on_connect
kite_ticker.on_close = on_close
kite_ticker.on_error = on_error

In [ ]:
# Establish KiteTicker connection
kite_ticker.connect(threaded=True)